# Computergestützte Analyse von Fanfiction auf Archive of Our Own

## Eine korpusbasierte Frequenz-, Topic- und Sentimentanalyse themenspezifischer Diskurse

Dieses Notebook wertet als CSV bereitgestellte Fanfiction-Daten der Plattform AO3 aus und ermöglicht die Untersuchung themenspezifischer Diskurse. Über die Bereitstellung eines Untersuchungsgegenstandes und einer Vergleichsliste kann es grundsätzlich für unterschiedliche Forschungsfragen genutzt werden. 

Mit dem Notebook lassen sich sowohl die Texte als auch die Metadaten getrennt voneinander analysieren. Es fokussiert dabei auf inhaltliche Zusammenhänge und berücksichtigt keine zeitlichen Dimensionen. Um zeitliche Entwicklungen nachvollziehen zu können, muss die Analyse daher jeweils getrennt für die einzelnen Zeiträume durchgeführt werden.

Konkret wird untersucht:
- Häufigkeit: Wie häufig werden über die Vergleichsliste definierte Themen aufgegriffen?
- Thematische Einbettung: Welche Bedeutung haben diese Themen am Gesamt-Topic-Spektrum?
- Valenz: Werden diese Themen überwiegend positiv, neutral oder negativ dargestellt?

**Zentrale Annahmen** sind:
- Fanfiction ist kein neutraler Raum, sondern reagiert auf externe Diskurse
- AO3 Tags sind soziale Metadaten und spiegeln bewusste Selbstverortung der Autor*innen wieder
- Textinhalte können implizite Repräsentationen enthalten, die nicht explizit getaggt sind

**Vorgehen**:
1. Allgemeines Setup und Einstellungen
2. Einlesen und Aufbereiten des Untersuchungsgegenstands
3. Einlesen von Vergleichsliste, Charakter-Liste und Stopwords
4. Berechnung eines themenspezifischen Likelihood-Scores
5. Allgemeine statistische Auswertung
6. Topic Modeling zur Einbettung des Themas im Gesamtkontext
7. Sentimentanalyse zur Bewertung des Themas in einschlägigen Texten


### 1. Allgemeines Setup und Einstellungen
**Zelle 1:** Installtion/Upgrade der erforderlichen Bibliotheken & Pakete *(optional)*  
**Zelle 2:** Importieren der erforderlichen Bibliotheken & Pakete  
**Zelle 3:** Definieren von Input und Output  

In [1]:
# Sofern erforderlich: Installation/Upgrade der nachfolgenden Bibliotheken
!uv pip install --upgrade scikit-learn
!uv pip install --upgrade lxml
!uv pip install --upgrade nltk
!uv pip install --upgrade seaborn
!uv pip install --upgrade wordcloud

Resolved 5 packages in 208ms
Prepared 1 package in 2ms
Uninstalled 1 package in 242ms
Installed 1 package in 819ms
 - numpy==2.4.0
 + numpy==2.4.1
Resolved 1 package in 65ms
Audited 1 package in 0.50ms
Resolved 6 packages in 83ms
Audited 6 packages in 1ms
Resolved 15 packages in 150ms
Prepared 1 package in 1ms
Uninstalled 1 package in 31ms
Installed 1 package in 96ms
 - pillow==12.0.0
 + pillow==12.1.0
Resolved 12 packages in 134ms
Audited 12 packages in 1ms


In [2]:
# --- Standard Library ---
import os
import re
import csv
import math
import unicodedata
from datetime import datetime
from itertools import combinations
from collections import Counter, defaultdict
import xml.etree.ElementTree as ET

# --- Third-party Libraries: Data & Math ---
import numpy as np
import pandas as pd

# --- NLP & Text Processing ---
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

# --- Machine Learning / Topic Modeling ---
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer,
    ENGLISH_STOP_WORDS
)
from sklearn.decomposition import LatentDirichletAllocation

# --- Visualization ---
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud


In [43]:
# Input
# - Pfad derjenigen Datei, die die Texte der Fanfiction enthält:
TEXT_CSV = "Ausgabedokumente/AO3_HP_FanFic_10_2025_text.csv"
# - Pfad derjenigen Datei, die die Metadaten wie Title, Rating, Category, Tags enthält:
META_CSV = "Ausgabedokumente/AO3_HP_FanFic_10_2025_meta.csv"

# Output
OUTPUT_PREFIX = "Analysis_HP_FanFic_10_2025"
OUTPUT_FOLDER = "Analysedokumente"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# - Anlegen des Analyse-Dokuments
ANALYSIS_OUTPUT_PATH = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_analyse.txt"
)

if os.path.exists(ANALYSIS_OUTPUT_PATH):
    raise RuntimeError(
        f"Analyse-Datei existiert bereits:\n{ANALYSIS_OUTPUT_PATH}\n"
        "Bitte umbenennen oder löschen."
    )

def now_ts():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
with open(ANALYSIS_OUTPUT_PATH, "w", encoding="utf-8") as f:
    f.write("=" * 60 + "\n")
    f.write(f"{OUTPUT_PREFIX} – Gesamtanalyse\n")
    f.write(f"Erstellt am: {now_ts()}\n")
    f.write("=" * 60 + "\n\n")

def write_analysis_block(title, lines, also_print=True):
    ts = now_ts()
    if isinstance(lines, str):
        lines = [lines]
    block = []
    block.append(f"[{ts}] {title}")
    block.append("-" * 60)
    block.extend(lines)
    block.append("")
    text = "\n".join(block)
    if also_print:
        print(text)
    with open(ANALYSIS_OUTPUT_PATH, "a", encoding="utf-8") as f:
        f.write(text + "\n")

### 2. Einlesen der Daten, Textnormalisierung und Tag-Bereinigung

**Zelle 1:** Einlesen und Aufbereiten des Untersuchungsgegenstandes  
**Zelle 2:** Prüfen der geladenen Datenstruktur *(optional)*

In [44]:
# CSV einlesen
df_text = pd.read_csv(TEXT_CSV, encoding='utf-8-sig', quotechar='"')
df_meta = pd.read_csv(META_CSV, encoding='utf-8-sig', quotechar='"')

# Zusammenführung über WorkID
df = df_meta.merge(df_text, on="work_id", how="left")
df = df.fillna("NaN")

# Text normalisieren und lemmatisieren
nltk.download("wordnet")
lemmatizer = WordNetLemmatizer()

def normalize_and_lemmatize(text):
    if pd.isna(text):
        return ""
    # Unicode-Normalisierung
    text = unicodedata.normalize("NFKD", text)
    # Kleinbuchstaben
    text = text.lower()
    # Sonderzeichen entfernen
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    # Mehrere Leerzeichen reduzieren
    text = re.sub(r"\s+", " ", text).strip()
    # Lemmatisierung
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(tokens)

df["text_norm_lem"] = df["text"].apply(normalize_and_lemmatize)


def prepare_text_for_topics(text):
    if not text:
        return ""
    # Zahlen entfernen
    text = re.sub(r"\d+", " ", text)
    # sehr kurze Wörter entfernen (<=2 Zeichen)
    text = re.sub(r"\b\w{1,2}\b", " ", text)
    # Mehrfach-Leerzeichen
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["text_topic"] = df["text_norm_lem"].apply(prepare_text_for_topics)


# Tag-Aufbereitung
TAG_COLS = ["category", "relationship", "character", "freeform"]

def split_tags(cell):
    if not cell:
        return []
    return [t.strip().lower() for t in str(cell).split(",") if t.strip()]

for col in TAG_COLS:
    df[col] = df[col].apply(split_tags)

def normalize_tag(tag):
    tag = tag.lower().strip()
    tag = tag.replace(" ", "_")
    tag = tag.replace("-", "_")
    return tag

# --- NaN aus Freeform und Character-Tags entfernen ---
for col in ["freeform", "character"]:
    df[col] = df[col].apply(lambda tags: [normalize_tag(t) for t in tags if t not in [None, np.nan, "nan", "NaN"]])

df["rating"] = df["rating"].str.strip().str.lower()
df["num_characters"] = df["character"].apply(len)
df["num_freeform_tags"] = df["freeform"].apply(len)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Uni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
analysis_lines = []

# Umfang und Struktur des Datensatzes
analysis_lines.append(f"\nDer Datensatz umfasst {len(df)} Werke und ist strukturiert nach:")
analysis_lines.append(", ".join(df.columns.tolist()) )

# Zentral ausgeben + protokollieren
write_analysis_block(
    title="Datenprüfung – Umfang, Struktur, Darstellung",
    lines=analysis_lines
)

# Zugriff prüfen:
print("-"*30, " nur im Notebook ", "-"*30)
print("Datenzugriff prüfen")
display(df.head(1))
print(df["text_norm_lem"].iloc[0][:80])
print("-"*30, " nur im Notebook ", "-"*30)

[2026-01-10 22:39:24] Datenprüfung – Umfang, Struktur, Darstellung
------------------------------------------------------------

Der Datensatz umfasst 2111 Werke und ist strukturiert nach:
work_id, title, rating, category, relationship, character, freeform, text, text_norm_lem, text_topic, num_characters, num_freeform_tags

------------------------------  nur im Notebook  ------------------------------
Datenzugriff prüfen


,work_id,title,rating,category,relationship,character,freeform,text,text_norm_lem,text_topic,num_characters,num_freeform_tags
0,76158526,Will Harry be able to walk ever again?,explicit,[m/m],[harry potter/charlie weasley],"[harry_potter, charlie_weasley]","[sexual_overstimulation, plot_what_plot/porn_w...","""Come Harry, let's go home."" Ms. Weasley takes...",come harry let s go home m weasley take me tog...,come harry let home weasley take together with...,2,5


come harry let s go home m weasley take me together with all the other weasleys 
------------------------------  nur im Notebook  ------------------------------


### 3. Einlesen von Vergleichsliste, Characterliste und Stopwords
**Zelle 1:** Laden der Vergleichsliste aus dem Ordner "HilfsDokumente"   
**Zelle 2:** Test der Wortliste *(optional)*

In [46]:
# Pfade zu den Dateien
comparison_file = "HilfsDokumente/compare_list.txt"
character_file = "HilfsDokumente/character_list.txt"
stopword_file = "HilfsDokumente/stopword_list.txt"

analysis_lines = []

# --- Funktion zum Einlesen und Prüfen einer Liste ---
def load_list(file_path, list_name):
    lines = []
    title = ""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            terms = set(line.strip().lower() for line in f if line.strip())

        if not terms:
            raise ValueError(f"{list_name} wurde gelesen, enthält aber keine gültigen Terme.")

        terms_sorted = sorted(terms, key=len, reverse=True)

        lines.append(f"{list_name} erfolgreich eingelesen.")
        lines.append(f"Dateipfad: {file_path}")
        lines.append(f"Umfang der Liste ({list_name}): {len(terms_sorted)}")

        # Vorschau
        preview_n = 10
        lines.append("")
        lines.append(f"Auszug aus {list_name} (erste {preview_n} Terme):")
        lines.append(", ".join(terms_sorted[:preview_n]))
        lines.append("")

        title = f"{list_name} – erfolgreich geladen"
        return terms_sorted, lines, title

    except FileNotFoundError:
        lines.append(f"FEHLER: {list_name} konnte nicht gefunden werden.")
        lines.append(f"Erwarteter Pfad: {file_path}")
        lines.append("")
        title = f"{list_name} – FEHLER (Datei nicht gefunden)"
        return [], lines, title

    except Exception as e:
        lines.append(f"FEHLER beim Einlesen von {list_name}:")
        lines.append(str(e))
        lines.append("")
        title = f"{list_name} – FEHLER (Einlesen fehlgeschlagen)"
        return [], lines, title


# --- Vergleichsliste einlesen ---
compare_list, compare_lines, compare_title = load_list(comparison_file, "Vergleichsliste")
analysis_lines.extend(compare_lines)

# --- Character-Liste einlesen ---
character_list, character_lines, character_title = load_list(character_file, "Character-Liste")
analysis_lines.extend(character_lines)

# --- Stopword-Liste einlesen ---
stopword_list, stopword_lines, stopword_title = load_list(stopword_file, "Stopword-Liste")
analysis_lines.extend(stopword_lines)

# --- Zentral ausgeben + protokollieren ---
write_analysis_block(
    title="Listen Einlesen – Übersicht",
    lines=analysis_lines
)


[2026-01-10 22:39:24] Listen Einlesen – Übersicht
------------------------------------------------------------
Vergleichsliste erfolgreich eingelesen.
Dateipfad: HilfsDokumente/compare_list.txt
Umfang der Liste (Vergleichsliste): 139

Auszug aus Vergleichsliste (erste 10 Terme):
trans exclusionary radical feminist, gender recognition certificate, gender confirmation surgeries, person with a trans history, assigned female at birth, transgender individuals, transgender individual, assigned male at birth, gender non confirming, gender non conformity

Character-Liste erfolgreich eingelesen.
Dateipfad: HilfsDokumente/character_list.txt
Umfang der Liste (Character-Liste): 195

Auszug aus Character-Liste (erste 10 Terme):
gregorowitsch, crookshanks, shacklebolt, xenophilius, grindelwald, mcgonagall, ollivander, nymphadora, longbottom, scrimgeour

Stopword-Liste erfolgreich eingelesen.
Dateipfad: HilfsDokumente/stopword_list.txt
Umfang der Liste (Stopword-Liste): 178

Auszug aus Stopword-Liste

In [47]:
# Test der Vergleichsliste
# - TestText
text1 = """
Transgender is an adjective to describe people whose gender identity differs
from the sex they were assigned at birth. 
People who are transgender may also use other terms, in addition to transgender, 
to describe their gender more specifically. 
Use the term(s) the person uses to describe their gender. 
It is important to note that being transgender is not dependent upon 
physical appearance or medical procedures. 
A person can call themself transgender the moment they realize 
that their gender identity is different than the sex they were assigned at birth.
"""

text1_lower = text1.lower()
matches = []
matches_counter = Counter()

# - Greedy-Matching
for term in compare_list:
    term_escaped = re.escape(term)
    # Alle Vorkommen zählen
    found = re.findall(rf'\b{term_escaped}\b', text1_lower)
    if found:
        matches_counter[term] += len(found)
        # Term aus Text entfernen, um Doppelzählungen zu vermeiden
        text1_lower = re.sub(rf'\b{term_escaped}\b', ' ', text1_lower)

# - Ausgabe
print("Gefundene Begriffe mit Häufigkeit:")
for term, count in matches_counter.items():
    print(f"{term}: {count}")

Gefundene Begriffe mit Häufigkeit:
assigned at birth: 2
gender identity: 2
transgender: 5


### 4. Berechnung eines Likelihood-Score
Um die Intensität der thematischen Auseinandersetzung differenziert abzubilden, wird ein kontinuierlicher Likelihood-Score eingeführt. Dieser berücksichtigt die Anzahl und Häufigkeit der in der Vergleichsliste enthaltenen Begriffe und modelliert den thematischen Bezug als graduelles Merkmal. 

#### Technische Beschreibung des Likelihood-Scores
Der Likelihood-Score ist ein gewichteter Relevanzwert, der angibt, wie stark ein Fanfiction-Werk auf eine vordefinierte Vergleichsliste von Begriffen (z. B. trans-spezifische Terme) reagiert. Der Score kombiniert Treffer in Metadaten-Tags und im Textkörper, um eine differenzierte Einschätzung zu ermöglichen.  

**Identifikation der Treffer**
Metadaten (freeform + character) und lemmatisierte Textkörper werden mit der Vergleichsliste abgeglichen. Die Vergleichsliste ist dazu absteigend nach Ausdruckslänge sortiert, sodass zusammenhängende Ausdrücke korrekt identifiziert werden können. Nach einem Treffer wird der entsprechende Textabschnitt maskiert, um Mehrfachzählungen derselben Textstelle und damit eine künstliche Erhöhung des Scores zu vermeiden. Die Trefferzahl wird pro Term erfasst, um Aussagen darüber treffen zu können, welche Begriffe wie häufig verwendet werden.  

**Berechnung der Term-Scores**
Bei der Berechnung des Scores soll berücksichtigt werden, dass viele unterschiedliche relevante Begriffen eine breitere, eine wiederholte Verwendung relevanter Begriffe eine tiefere Auseinandersetzung mit einer Thematik indizieren. Daher wird für jeden Bereich (freeform, character, text) ein Score gebildet, der zwei Komponenten kombiniert: Unique Hits gibt an, wie viele unterschiedliche Terme aus der Vergleichsliste im Text vorkommen, Frequency Sum enthält die standardmäßig logarithmisch skaliert Trefferanzahl pro Term.  

**Gewichtung der Bereiche**
Da Tags regelmäßig prägnanter und oft genauer in der thematischen Einordnung sind, während der Text länger, aber potenziell verrauschter ist, werden die Scores der Tags und des Textkörpers mit 80:20 gewichtet kombiniert.  

#### Aufbau
**Zelle 1:** Gewichtung *(einstellbar)*  
**Zelle 2:** Berechnung des Likeliehood-Scores  
**Zelle 3:** Ausgabe zentraler Kennzahlen

In [48]:
# Gewichtung EINSTELLBAR!
WEIGHTS = {"tags": 0.8, "text": 0.2} 
USE_LOG_SCALING = True 

In [49]:
# --- Matches identifizieren und zählen mit Greedy-Matching gegen compare_list ---
def count_term_hits(text, terms):
    counter = Counter()

    if not text:
        return counter

    # Alles klein, Unterstriche und Bindestriche normalisieren
    text_lower = text.lower()
    text_normalized = text_lower.replace("_", " ").replace("-", " ")

    # Maske zur Vermeidung doppelter Treffer (Greedy)
    mask = [" "] * len(text_normalized)

    for term in terms:
        term_lower = term.lower()
        term_normalized = term_lower.replace("_", " ").replace("-", " ")
        pattern = re.escape(term_normalized)

        # Regex für ganze Wörter / Phrasen
        regex = re.compile(rf'\b{pattern}\b')

        for m in regex.finditer(text_normalized):
            start, end = m.span()

            # Prüfen, ob Bereich bereits maskiert wurde
            if all(c == " " for c in mask[start:end]):
                counter[term] += 1
                # Bereich maskieren
                mask[start:end] = ["*"] * (end - start)

    return counter


# --- Score berechnen ---
def score(row):
    # --- Freeform Tags ---
    freeform_text = " ".join(row.get("freeform", []))
    freeform_hits = count_term_hits(freeform_text, compare_list)
    freeform_unique = len(freeform_hits)
    freeform_freq_sum = (
        sum(math.log(1 + v) for v in freeform_hits.values()) if USE_LOG_SCALING else sum(freeform_hits.values())
    )
    freeform_score = freeform_unique + freeform_freq_sum

    # --- Character Tags ---
    char_text = " ".join(row.get("character", []))
    char_hits = count_term_hits(char_text, compare_list)
    char_unique = len(char_hits)
    char_freq_sum = (
        sum(math.log(1 + v) for v in char_hits.values()) if USE_LOG_SCALING else sum(char_hits.values())
    )
    char_score = char_unique + char_freq_sum

    # --- Text ---
    text_text = row.get("text_norm_lem", "")
    text_hits = count_term_hits(text_text, compare_list)
    text_unique = len(text_hits)
    text_freq_sum = (
        sum(math.log(1 + v) for v in text_hits.values()) if USE_LOG_SCALING else sum(text_hits.values())
    )
    text_score = text_unique + text_freq_sum

    # --- Gesamt-Score ---
    total_score = round(
        (WEIGHTS["tags"] * (freeform_score + char_score) + WEIGHTS["text"] * text_score),
        2
    )

    return pd.Series({
        "likelihood": total_score,
        "freeform_hits_counter": freeform_hits,
        "character_hits_counter": char_hits,
        "text_hits_counter": text_hits,
        "freeform_score": freeform_score,
        "character_score": char_score,
        "text_score": text_score
    })


# --- Score-Funktion auf DataFrame anwenden ---
df[[
    "likelihood",
    "freeform_hits_counter",
    "character_hits_counter",
    "text_hits_counter",
    "freeform_score",
    "character_score",
    "text_score"
]] = df.apply(score, axis=1)


# --- Trefferzahlen berechnen ---
df["freeform_hits_count"] = df["freeform_hits_counter"].apply(lambda c: sum(c.values()))
df["character_hits_count"] = df["character_hits_counter"].apply(lambda c: sum(c.values()))
df["text_hits_count"] = df["text_hits_counter"].apply(lambda c: sum(c.values()))
df["tag_hits_count"] = df["freeform_hits_count"] + df["character_hits_count"]
df["total_hits_count"] = df["freeform_hits_count"] + df["character_hits_count"] + df["text_hits_count"]

In [50]:
# --- Grundverteilung beschreiben ---
likelihood_desc = df["likelihood"].describe()
min_score = likelihood_desc["min"]
max_score = likelihood_desc["max"]
mean_score = round(likelihood_desc["mean"], 2)
median_score = likelihood_desc["50%"]
std_score = round(likelihood_desc["std"], 2)

# --- Grafische Darstellung der Grundverteilung ---
hist_file = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_graph_likelihood_distribution.png"
)

plt.figure(figsize=(8,5))
plt.hist(df["likelihood"], bins=30, color="skyblue", edgecolor="black")
plt.xlabel("Likelihood-Score")
plt.ylabel("Anzahl Texte")
plt.title(f"Verteilung des Likelihood-Scores für {OUTPUT_PREFIX}")
plt.tight_layout()
plt.savefig(hist_file)
plt.close()  

# --- Ausgabe in CSV-Datei zur manuellen Prüfung ---
df_export = df.copy()

df_export["freeform_hits_terms"] = df_export["freeform_hits_counter"].apply(
    lambda c: ", ".join(c.keys()) if c else ""
)
df_export["character_hits_terms"] = df_export["character_hits_counter"].apply(
    lambda c: ", ".join(c.keys()) if c else ""
)
df_export["text_hits_terms"] = df_export["text_hits_counter"].apply(
    lambda c: ", ".join(c.keys()) if c else ""
)

# Nur Zeilen mit mindestens einem Treffer
df_export = df_export[df_export["total_hits_count"] > 0]

output_columns = [
    "work_id",
    "likelihood",
    "freeform_hits_count",
    "freeform_hits_terms",
    "character_hits_count",
    "character_hits_terms",
    "text_hits_count",
    "text_hits_terms",
    "total_hits_count"
]

output_file = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_likelihood_positives.csv"
)

df_export[output_columns].to_csv(
    output_file,
    index=False,
    encoding="utf-8"
)

num_ids = len(df_export)

# --- Zusammenfassung für Analyse-Dokument ---
analysis_lines = []
analysis_lines.append("Score- und Treffer-Berechnung abgeschlossen für")
analysis_lines.append(f"Gewichtung: tags={WEIGHTS['tags']}, text={WEIGHTS['text']}")
analysis_lines.append(f"Log-Skalierung: {USE_LOG_SCALING}")
analysis_lines.append("")

analysis_lines.append(f"Anzahl der Werke mit mindestens einem Treffer: {num_ids}")

total_hits_sum = df_export["total_hits_count"].sum()
avg_hits_per_work = round(df_export["total_hits_count"].mean(), 2) if num_ids > 0 else 0
analysis_lines.append(f"Summe aller Treffer: {total_hits_sum}")
analysis_lines.append(f"Durchschnittliche Treffer pro Werk: {avg_hits_per_work}")
analysis_lines.append("")

analysis_lines.append("Likelihood-Score Verteilung:")
analysis_lines.append(f"Höchster Score: {max_score}")
analysis_lines.append(f"Niedrigster Score: {min_score}")
analysis_lines.append(f"Durchschnitt (Mean): {mean_score}")
analysis_lines.append(f"Median: {median_score}")
analysis_lines.append(f"Standardabweichung: {std_score}")
analysis_lines.append("")
analysis_lines.append(f"Histogramm des Likelihood-Scores gespeichert unter: {hist_file}")
analysis_lines.append("")
analysis_lines.append(f"Pfad zur CSV-Datei für die manuelle Prüfung: {output_file}")
analysis_lines.append("")

# --- Zentral ausgeben + protokollieren ---
write_analysis_block(
    title="Likelihood-Score - Gewichtung und Berechnung",
    lines=analysis_lines
)


[2026-01-10 22:41:34] Likelihood-Score - Gewichtung und Berechnung
------------------------------------------------------------
Score- und Treffer-Berechnung abgeschlossen für
Gewichtung: tags=0.8, text=0.2
Log-Skalierung: True

Anzahl der Werke mit mindestens einem Treffer: 98
Summe aller Treffer: 200
Durchschnittliche Treffer pro Werk: 2.04

Likelihood-Score Verteilung:
Höchster Score: 6.28
Niedrigster Score: 0.0
Durchschnitt (Mean): 0.07
Median: 0.0
Standardabweichung: 0.39

Histogramm des Likelihood-Scores gespeichert unter: Analysedokumente\Analysis_HP_FanFic_10_2025_graph_likelihood_distribution.png

Pfad zur CSV-Datei für die manuelle Prüfung: Analysedokumente\Analysis_HP_FanFic_10_2025_likelihood_positives.csv




### 5. Allgemeine Statistische Auswertung
Für die weitere Auswertung wird zunächst auf Basis der Grundverteilung des Likeliehood-Scores ein Schwellenwert gesetzt, der mit zusätzlichen Vorgaben hinsichtlich der Matches in den Tags und Textkörpern ergänzt werden kann.

Die vor dem Hintergrund dieses Schwellenwertes als relevant klassifizierten Werke werden in diesem Abschnitt näher untersucht hinsichtlich
- **Breite & Tiefe**: Wie viele Begriffe der Vergleichsliste werden verwendet (Breite) und wie häufig kommen einzelne Begriffe vor (Tiefe).
- **Rating**: Von Autor:innen vergeben (Explicit, General Audiences, Mature, Not Rated, Teen and Up). Ziel war zu prüfen, ob transbezogene Inhalte mit sexualisierten Inhalten oder Altersbeschränkungen assoziiert sind, bzw. ob transbezogene Inhalte auch in Werken ohne Altersbeschränkung vorkommen.  
- **Charakter**: Häufigkeit der Charaktere in relevanten Werken und Gesamt-Korpus, um die Relevanzrelation pro Figur zu bestimmen.  
Die Auswertung liefert so quantitative Hinweise darauf, welche Begriffe, Tags, Charaktere und Alterskategorien im Zusammenhang mit einschlägigen Werken besonders häufig auftreten.

#### Aufbau
**Zelle 1:** Definition eines Schwellenwertes (Likeliehood-Score)  
**Zelle 2:** Grundlegende Kennzahlen  
**Zelle 3:** Breite und Tiefe der Matches  
**Zelle 4:** Matches vs. Rating  
**Zelle 5:** Matches vs. Charactere

In [51]:
# Definition eines Schwellenwerts / Einstellbar!
LIKELIHOOD_THRESHOLD = 0.15
MIN_TAG_HITS = 0
MIN_TEXT_HITS = 0

df["is_relevant"] = (
    (df["likelihood"] >= LIKELIHOOD_THRESHOLD) &
    (df["tag_hits_count"] >= MIN_TAG_HITS) &
    (df["text_hits_count"] >= MIN_TEXT_HITS)
)

In [52]:
# Grundlegende Kennzahlen
analysis_lines = []

analysis_lines.append(f"Definierter Likelihood-Schwellenwert: {LIKELIHOOD_THRESHOLD}")
analysis_lines.append(f"Mindestanzahl Matches in Tags: {MIN_TAG_HITS}")
analysis_lines.append(f"Mindestanzahl Matches in Text: {MIN_TEXT_HITS}")
analysis_lines.append("")
analysis_lines.append(f"Anzahl aller Werke: {len(df)}")
analysis_lines.append(f"Anzahl einschlägiger Werke: {df['is_relevant'].sum()}")
analysis_lines.append(f"Anteil einschlägiger Werke: {round(df['is_relevant'].mean() * 100, 2)} %")
analysis_lines.append("")

df_relevant = df[df["is_relevant"]].copy()

# --- Freeform-Tags ---
# Anzahl Werke mit mindestens einem Freeform-Tag
relevant_freeform_works = (df_relevant["freeform"].apply(len) > 0).sum()
all_freeform_works = (df["freeform"].apply(len) > 0).sum()

# Gesamtanzahl Freeform-Tags
total_freeform_relevant = df_relevant["freeform"].apply(len).sum()
total_freeform_all = df["freeform"].apply(len).sum()

analysis_lines.append("Freeform-Tags:")
analysis_lines.append(
    f"- Werke mit mindestens einem Freeform-Tag: "
    f"{relevant_freeform_works} von {len(df_relevant)} "
    f"({round(100*relevant_freeform_works/len(df_relevant),1)} %)"
)
analysis_lines.append(
    f"- Gesamtanzahl Freeform-Tags in relevanten Werken: {total_freeform_relevant}"
)
analysis_lines.append(
    f"- Anteil dieser Freeform-Tags am gesamten Korpus: "
    f"{round(100*total_freeform_relevant/total_freeform_all,1)} %"
)
analysis_lines.append("")

# --- Character-Tags ---
relevant_character_works = (df_relevant["character"].apply(len) > 0).sum()
all_character_works = (df["character"].apply(len) > 0).sum()

total_character_relevant = df_relevant["character"].apply(len).sum()
total_character_all = df["character"].apply(len).sum()

analysis_lines.append("Character-Tags:")
analysis_lines.append(
    f"- Werke mit mindestens einem Character-Tag: "
    f"{relevant_character_works} von {len(df_relevant)} "
    f"({round(100*relevant_character_works/len(df_relevant),1)} %)"
)
analysis_lines.append(
    f"- Gesamtanzahl Character-Tags in relevanten Werken: {total_character_relevant}"
)
analysis_lines.append(
    f"- Anteil dieser Character-Tags am gesamten Korpus: "
    f"{round(100*total_character_relevant/total_character_all,1)} %"
)
analysis_lines.append("")

# Text-Statistiken
df["text_length"] = df["text_norm_lem"].apply(lambda x: len(x.split()))
df_relevant["text_length"] = df_relevant["text_norm_lem"].apply(
    lambda x: len(x.split())
)

total_text_length_relevant = df_relevant["text_length"].sum()
total_text_length_all = df["text_length"].sum()

analysis_lines.append("Textumfang der einschlägigen Werke:")
analysis_lines.append(
    f"- Gesamtlänge der Texte einschlägiger Werke: "
    f"{total_text_length_relevant:,} Tokens"
)
analysis_lines.append(
    f"- Anteil am gesamten Textkorpus: "
    f"{round(100 * total_text_length_relevant / total_text_length_all, 1)} %"
)
analysis_lines.append("")

# --- In Analyse-Dokument schreiben ---
write_analysis_block(
    title=f"Grundlegende Kennzahlen zu den einschlägigen Works",
    lines=analysis_lines
)

[2026-01-10 22:41:34] Grundlegende Kennzahlen zu den einschlägigen Works
------------------------------------------------------------
Definierter Likelihood-Schwellenwert: 0.15
Mindestanzahl Matches in Tags: 0
Mindestanzahl Matches in Text: 0

Anzahl aller Werke: 2111
Anzahl einschlägiger Werke: 98
Anteil einschlägiger Werke: 4.64 %

Freeform-Tags:
- Werke mit mindestens einem Freeform-Tag: 98 von 98 (100.0 %)
- Gesamtanzahl Freeform-Tags in relevanten Werken: 1536
- Anteil dieser Freeform-Tags am gesamten Korpus: 6.3 %

Character-Tags:
- Werke mit mindestens einem Character-Tag: 97 von 98 (99.0 %)
- Gesamtanzahl Character-Tags in relevanten Werken: 385
- Anteil dieser Character-Tags am gesamten Korpus: 5.4 %

Textumfang der einschlägigen Werke:
- Gesamtlänge der Texte einschlägiger Werke: 647,662 Tokens
- Anteil am gesamten Textkorpus: 10.4 %




In [60]:
# Breite & Tiefe der Matches – Statistik (compare_list)
analysis_lines = []

# --- Einschlägige Werke filtern ---
df_relevant = df[df["is_relevant"]].copy()

# --- Hilfsfunktionen ---
from collections import Counter

def aggregate_counters(series):
    """Alle Counter in einer Serie zusammenführen"""
    total = Counter()
    for c in series:
        total.update(c)
    return total

def works_with_hits(series_of_counters):
    """Zählt, in wie vielen Dokumenten mindestens ein Treffer vorhanden ist"""
    return sum(1 for c in series_of_counters if sum(c.values()) > 0)

# --- Counter pro Werk auf Basis compare_list ---
def get_compare_hits(row):
    """
    Zählt Treffer in freeform, character und text nur für compare_list.
    Verwendet dieselbe Greedy-Matching-Logik wie der Likelihood-Score
    (count_term_hits), um konsistente Ergebnisse sicherzustellen.
    """
    combined_counter = Counter()

    # --- Freeform ---
    freeform_tags = row.get("freeform", [])
    freeform_text = " ".join(freeform_tags)
    freeform_hits = count_term_hits(freeform_text, compare_list)
    combined_counter.update(freeform_hits)

    # --- Character ---
    char_tags = row.get("character", [])
    char_text = " ".join(char_tags)
    char_hits = count_term_hits(char_text, compare_list)
    combined_counter.update(char_hits)

    # --- Text ---
    text_content = row.get("text_norm_lem", "")
    text_hits = count_term_hits(text_content, compare_list)
    combined_counter.update(text_hits)

    return freeform_hits, char_hits, text_hits, combined_counter

# --- Counters erzeugen ---
df_relevant[[
    "freeform_hits_counter",
    "character_hits_counter",
    "text_hits_counter",
    "combined_counter"
]] = df_relevant.apply(
    lambda row: pd.Series(get_compare_hits(row)),
    axis=1
)

# --- Aggregierte Counter ---
counter_tags = aggregate_counters(
    df_relevant["freeform_hits_counter"] +
    df_relevant["character_hits_counter"]
)
counter_text = aggregate_counters(df_relevant["text_hits_counter"])
counter_combined = aggregate_counters(df_relevant["combined_counter"])

# --- Statistikfunktion ---
def term_stats(counter, series_of_counters, name):
    total_terms = sum(counter.values())
    unique_terms = len(counter)
    works_count = works_with_hits(series_of_counters)
    top_terms = counter.most_common(10)

    analysis_lines.append(f"Für {name}:")
    analysis_lines.append(f"Gesamtanzahl Treffer: {total_terms} in {works_count} Werken")
    analysis_lines.append(f"Anzahl verschiedener Begriffe: {unique_terms}")
    analysis_lines.append(f"Top 10 Begriffe: {top_terms}")
    analysis_lines.append("")

# --- Auswertungen ---
term_stats(
    counter_tags,
    df_relevant["freeform_hits_counter"] +
    df_relevant["character_hits_counter"],
    "Tags (Freeform + Character, compare_list)"
)
term_stats(
    counter_text,
    df_relevant["text_hits_counter"],
    "Textkörper (compare_list)"
)

# --- Co-Occurrence Matrix (Top 20 kombinierte Begriffe) ---
top_n = 20
top_terms_list = [t for t, _ in counter_combined.most_common(top_n)]

co_matrix = pd.DataFrame(
    0,
    index=top_terms_list,
    columns=top_terms_list
)

for counter in df_relevant["combined_counter"]:
    terms_in_doc = {t: counter[t] for t in counter if t in top_terms_list}
    for t1, count1 in terms_in_doc.items():
        co_matrix.loc[t1, t1] += count1
        for t2, count2 in terms_in_doc.items():
            if t1 != t2:
                co_matrix.loc[t1, t2] += min(count1, count2)

analysis_lines.append("Co-Occurrence Top Begriffe (compare_list):")
analysis_lines.append(co_matrix.to_string())
analysis_lines.append("")

# --- Heatmap ---
co_matrix_file = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_graph_cooccurrence_compare.png"
)

plt.figure(figsize=(10, 8))
sns.heatmap(co_matrix, cmap="viridis", annot=True, fmt="d")
plt.title(f"Co-Occurrence Top 20 Begriffe (compare_list) für {OUTPUT_PREFIX}")
plt.tight_layout()
plt.savefig(co_matrix_file)
plt.close()

analysis_lines.append(f"Grafik zur Co-Occurrence gespeichert unter: {co_matrix_file}")

# --- Wordcloud ---
wordcloud_file = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_graph_wordcloud_compare.png"
)

wc = WordCloud(
    width=800,
    height=400,
    background_color="white"
).generate_from_frequencies(counter_combined)

plt.figure(figsize=(12, 6))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title(f"Wordcloud für compare_list – {OUTPUT_PREFIX}")
plt.tight_layout()
plt.savefig(wordcloud_file)
plt.close()

analysis_lines.append(f"Grafik Wordcloud gespeichert unter: {wordcloud_file}")

# --- Histogramm: Tags vs. Text pro Werk ---
hist_file_tags_text = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_graph_hits_tags_vs_text_compare.png"
)

# Berechnung der Treffer pro Werk
tags_counts = (
    df_relevant["freeform_hits_counter"].apply(lambda c: sum(c.values())) +
    df_relevant["character_hits_counter"].apply(lambda c: sum(c.values()))
)
text_counts = df_relevant["text_hits_counter"].apply(lambda c: sum(c.values()))
work_labels = df_relevant["work_id"].astype(str)

plt.figure(figsize=(12, 6))
plt.bar(work_labels, tags_counts, label="Tags (Freeform + Character)")
plt.bar(work_labels, text_counts, bottom=tags_counts, label="Text")
plt.xticks(rotation=90)
plt.xlabel("Werk-ID")
plt.ylabel("Anzahl Treffer")
plt.title(f"Art der Matches pro Werk (compare_list) für {OUTPUT_PREFIX}")
plt.legend()
plt.tight_layout()
plt.savefig(hist_file_tags_text)
plt.close()

analysis_lines.append(
    f"Grafik zu der Art der Matches pro Werk gespeichert unter: {hist_file_tags_text}"
)

# --- Histogramm: Gesamt-Treffer pro Werk ---
hist_file_total = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_graph_hits_per_work_compare.png"
)

total_hits_per_work = df_relevant["combined_counter"].apply(lambda c: sum(c.values()))

plt.figure(figsize=(8, 5))
plt.hist(total_hits_per_work, bins=30, edgecolor="black")
plt.xlabel("Gesamtanzahl Treffer pro Werk")
plt.ylabel("Anzahl Werke")
plt.title("Verteilung der Treffer pro Werk (compare_list)")
plt.tight_layout()
plt.savefig(hist_file_total)
plt.close()

analysis_lines.append(
    f"Grafik zur Verteilung der Matches nach Werken gespeichert unter: {hist_file_total}"
)

# --- Analyseblock schreiben ---
write_analysis_block(
    title="Breite & Tiefe der Matches – Statistik (compare_list)",
    lines=analysis_lines
)


[2026-01-10 23:07:50] Breite & Tiefe der Matches – Statistik (compare_list)
------------------------------------------------------------
Für Tags (Freeform + Character, compare_list):
Gesamtanzahl Treffer: 101 in 59 Werken
Anzahl verschiedener Begriffe: 11
Top 10 Begriffe: [('trans', 72), ('transphobia', 5), ('pronouns', 5), ('ftm', 4), ('genderfluid', 4), ('non binary', 4), ('gender dysphoria', 3), ('gender euphoria', 1), ('terf', 1), ('genderqueer', 1)]

Für Textkörper (compare_list):
Gesamtanzahl Treffer: 99 in 55 Werken
Anzahl verschiedener Begriffe: 24
Top 10 Begriffe: [('transition', 32), ('binder', 13), ('transgender', 12), ('trans', 11), ('sexual orientation', 3), ('transitioning', 3), ('non binary', 3), ('deadname', 2), ('intersex', 2), ('gender expression', 2)]

Co-Occurrence Top Begriffe (compare_list):
                     trans  transition  binder  transgender  non binary  transphobia  pronouns  ftm  genderfluid  gender dysphoria  sexual orientation  transitioning  genderq

In [54]:
# Matches vs Rating
analysis_lines = []
analysis_lines.append("Matches nach Rating")
analysis_lines.append("- Analyse, in welchem Rating die Werke zum Themenschwerpunkt enthalten sind")
analysis_lines.append("")

# - Anzahl und Anteile einschlägiger Werke
rating_counts = df_relevant["rating"].value_counts()
rating_percent = (rating_counts / len(df_relevant) * 100).round(2)

analysis_lines.append("Anzahl einschlägiger Werke pro Rating:")
for r, count in rating_counts.items():
    analysis_lines.append(f"{r}: {count} Werke ({rating_percent[r]}%)")

# - Vergleich: Gesamt vs. einschlägig
rating_compare = pd.DataFrame({
    "gesamt": df["rating"].value_counts(normalize=True),
    "einschlägig": df_relevant["rating"].value_counts(normalize=True)
}).fillna(0)

analysis_lines.append("")
analysis_lines.append("Relativer Anteil im Vergleich zur Gesamtstichprobe (normiert):")
for r in rating_compare.index:
    gesamt_pct = round(rating_compare.loc[r, "gesamt"]*100, 2)
    relevant_pct = round(rating_compare.loc[r, "einschlägig"]*100, 2)
    analysis_lines.append(f"{r}: Gesamt {gesamt_pct}% | Einschlägig {relevant_pct}%")

analysis_lines.append("")

# Grafiken erstellen
import matplotlib.pyplot as plt
import seaborn as sns

# - Absolute Anzahl einschlägiger Werke pro Rating
fig1_file = os.path.join(OUTPUT_FOLDER, f"{OUTPUT_PREFIX}_graph_rating_counts.png")
plt.figure(figsize=(8,5))
sns.barplot(x=rating_counts.index, y=rating_counts.values, palette="viridis")
plt.ylabel("Anzahl Werke")
plt.xlabel("Rating")
plt.title(f"Einschlägige Werke pro Rating für {OUTPUT_PREFIX}")
plt.tight_layout()
plt.savefig(fig1_file)
plt.close()
analysis_lines.append(f"Einschlägige Werke pro Rating gespeichert unter: {fig1_file}")

# - Relativer Vergleich: Gesamt vs. einschlägig
fig2_file = os.path.join(OUTPUT_FOLDER, f"{OUTPUT_PREFIX}_graph_rating_comparison.png")
rating_compare_plot = rating_compare.sort_index()  # optional nach Rating sortieren
rating_compare_plot.plot(kind="bar", figsize=(8,5))
plt.ylabel("Anteil (normiert)")
plt.xlabel("Rating")
plt.title(f"Rating-Vergleich Gesamt vs. Einschlägig für {OUTPUT_PREFIX}")
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(fig2_file)
plt.close()
analysis_lines.append(f"Rating-Vergleich Gesamt vs. Einschlägig gespeichert unter: {fig2_file}")

# Ausgabeblock schreiben
write_analysis_block(
    title="Matches nach Rating",
    lines=analysis_lines
)


C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\2268886027.py:37: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=rating_counts.index, y=rating_counts.values, palette="viridis")


[2026-01-10 22:41:52] Matches nach Rating
------------------------------------------------------------
Matches nach Rating
- Analyse, in welchem Rating die Werke zum Themenschwerpunkt enthalten sind

Anzahl einschlägiger Werke pro Rating:
explicit: 50 Werke (51.02%)
teen and up audiences: 21 Werke (21.43%)
mature: 13 Werke (13.27%)
general audiences: 12 Werke (12.24%)
not rated: 2 Werke (2.04%)

Relativer Anteil im Vergleich zur Gesamtstichprobe (normiert):
explicit: Gesamt 38.7% | Einschlägig 51.02%
general audiences: Gesamt 24.63% | Einschlägig 12.24%
mature: Gesamt 10.75% | Einschlägig 13.27%
not rated: Gesamt 3.84% | Einschlägig 2.04%
teen and up audiences: Gesamt 22.07% | Einschlägig 21.43%

Einschlägige Werke pro Rating gespeichert unter: Analysedokumente\Analysis_HP_FanFic_10_2025_graph_rating_counts.png
Rating-Vergleich Gesamt vs. Einschlägig gespeichert unter: Analysedokumente\Analysis_HP_FanFic_10_2025_graph_rating_comparison.png



In [55]:
# Matches vs Character (verschärft: Vergleichsliste + Character-Liste)

analysis_lines = []
analysis_lines.append("Figuren im Themenzusammenhang- Analyse der Figuren in einschlägigen Werken")
analysis_lines.append("")
analysis_lines.append("")

# df_relevant und df_nonrelevant
df_relevant = df[df["is_relevant"]].copy()
df_nonrelevant = df[~df["is_relevant"]].copy()

from collections import defaultdict, Counter

# Figuren mit explizitem Bezug: nur Tags, die sowohl compare_list als auch character_list matchen
analysis_lines.append("Figuren mit explizitem Bezug")

combined_tags_counter = Counter()
combined_tags_work_ids = defaultdict(set) 

for idx, row in df_relevant.iterrows():
    work_id = row["work_id"]
    all_tags = row.get("freeform", []) + row.get("character", [])

    for tag in all_tags:
        # Normalisierung: Unterstriche / Leerzeichen
        tag_norm = tag.lower().replace("-", "_").replace(" ", "_")
        
        # Prüfen: mindestens ein Begriff aus compare_list UND ein Begriff aus character_list enthalten
        if any(term in tag_norm for term in compare_list) and any(char in tag_norm for char in character_list):
            combined_tags_counter[tag] += 1
            combined_tags_work_ids[tag].add(work_id)

# Top 15 kombinierte Tags
top_combined_tags = pd.DataFrame(
    [
        (tag, combined_tags_counter[tag], sorted(combined_tags_work_ids[tag]))
        for tag, _ in combined_tags_counter.most_common(15)
    ],
    columns=["tag", "count", "work_ids"]
)

# Ausgabe
analysis_lines.append("Top 15 Figuren mit explizitem Bezug (inkl. Work IDs):")
for idx, row in top_combined_tags.iterrows():
    work_ids_str = ", ".join(str(wid) for wid in row["work_ids"])
    analysis_lines.append(f"{idx+1}. {row['tag']} ({row['count']} Werke)")
    analysis_lines.append(f"Work IDs: {work_ids_str}")
analysis_lines.append("")

# Implizites Vorkommen
analysis_lines.append("Figuren mit implizitem Bezug (Vorkommen einschlägigen Texten)")
char_counter = Counter()
for chars in df_relevant["character"]:
    char_counter.update(chars)

top_characters = pd.DataFrame(
    char_counter.most_common(15),
    columns=["character", "count"]
)

analysis_lines.append("Top 15 Figuren in einschlägigen Werken:")
for idx, row in top_characters.iterrows():
    analysis_lines.append(f"{idx+1}. {row['character']}: {row['count']} Werke")
analysis_lines.append("")

# Relatives Vorkommen einschlägig vs. alle Werke
char_relevant = Counter()
char_nonrelevant = Counter()

for chars in df_relevant["character"]:
    char_relevant.update(chars)
for chars in df_nonrelevant["character"]:
    char_nonrelevant.update(chars)

comparison = []
for char in set(list(char_relevant.keys()) + list(char_nonrelevant.keys())):
    count_rel = char_relevant.get(char, 0)
    count_nonrel = char_nonrelevant.get(char, 0)
    if count_rel + count_nonrel >= 5:
        rel_fraction = count_rel / (count_rel + count_nonrel)
        comparison.append((char, count_rel, count_nonrel, round(rel_fraction, 3)))

comparison_df = pd.DataFrame(
    comparison, columns=["character", "relevant_count", "nonrelevant_count", "fraction_relevant"]
).sort_values("fraction_relevant", ascending=False)

analysis_lines.append("Relativer Anteil einschlägige vs. alle Werke (mind. 5 Werke insgesamt):")
for idx, row in comparison_df.head(15).iterrows():
    analysis_lines.append(
        f"{idx+1}. {row['character']}: {row['relevant_count']} / {row['nonrelevant_count']} Werke "
        f"({row['fraction_relevant']*100} % in relevanten Werken)"
    )

# Grafiken
fig_chars_file = os.path.join(OUTPUT_FOLDER, f"{OUTPUT_PREFIX}_graph_topcharacters.png")
plt.figure(figsize=(10,6))
sns.barplot(x="count", y="character", data=top_characters, palette="viridis")
plt.xlabel("Anzahl einschlägiger Werke")
plt.ylabel("Figur")
plt.title(f"Top 15 Figuren in einschlägigen Werken für {OUTPUT_PREFIX}")
plt.tight_layout()
plt.savefig(fig_chars_file)
plt.close()
analysis_lines.append(f"\nGrafik der Top Figuren gespeichert unter: {fig_chars_file}")

fig_chars_rel_file = os.path.join(OUTPUT_FOLDER, f"{OUTPUT_PREFIX}_graph_relativecharacterrelevance.png")
plt.figure(figsize=(10,6))
sns.barplot(x="fraction_relevant", y="character", data=comparison_df.head(15), palette="coolwarm")
plt.xlabel("Anteil in relevanten Werken")
plt.ylabel("Figur")
plt.title(f"Figuren-Anteil in einschlägigen vs. allen Texten für {OUTPUT_PREFIX}")
plt.xlim(0,1)
plt.tight_layout()
plt.savefig(fig_chars_rel_file)
plt.close()
analysis_lines.append(f"Grafik zum relativen Vorkommen (relevant vs. nicht relevant) gespeichert unter: {fig_chars_rel_file}")

# Ausgabeblock schreiben
write_analysis_block(
    title="Matches vs Character – kombiniert compare_list + character_list",
    lines=analysis_lines
)


C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\1609719855.py:97: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x="count", y="character", data=top_characters, palette="viridis")
C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\1609719855.py:108: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x="fraction_relevant", y="character", data=comparison_df.head(15), palette="coolwarm")


[2026-01-10 22:41:53] Matches vs Character – kombiniert compare_list + character_list
------------------------------------------------------------
Figuren im Themenzusammenhang- Analyse der Figuren in einschlägigen Werken


Figuren mit explizitem Bezug
Top 15 Figuren mit explizitem Bezug (inkl. Work IDs):
1. trans_regulus_black (30 Werke)
Work IDs: 60181408, 71115726, 71390066, 71427291, 71558041, 71638646, 71654156, 71677146, 71681946, 71749946, 71813036, 71837436, 71837786, 71945241, 71961046, 72023176, 72078311, 72092711, 72108571, 72287316, 72417596, 72500056, 72591531, 72629041, 72636416, 73011126, 73128221, 73189381, 73447521, 73454846
2. trans_sirius_black (4 Werke)
Work IDs: 71754236, 71908236, 72624701, 73051026
3. trans_percy_weasley (3 Werke)
Work IDs: 72023136, 73035356, 73404586
4. ftm_percy_weasley (3 Werke)
Work IDs: 72023136, 73035356, 73404586
5. genderfluid_sirius_black (3 Werke)
Work IDs: 72455821, 72528926, 73168956
6. trans_remus_lupin (3 Werke)
Work IDs: 71837436,

### 6. Topic Modeling zur Einbettung des Themas im Gesamtkontext

Im Rahmen dieses Projekts werden die Topic-Modeling-Analysen getrennt für **narrative Volltexte** und **paratextuelle Tags** durchgeführt, um die Haupt-Topics zu identifizieren. Zudem wird unterschieden zwischen: **allen Werken** und **einschlägigen Werken** (positiver Trans-Bezug),  
um zu prüfen, ob Trans-Themen tatsächlich dominieren und welche Topics damit verknüpft sind. Die vollständigen Topic-Zuordnungen werden im Ordner *Ausgabedokumente* gespeichert.

**Zelle 1:** Topic Modeling für alle Werke (Text)  
**Zelle 2:** Topic Modeling für einschlägige Werke (Text)  
**Zelle 3:** Topic Modeling für alle Werke (Tags)  
**Zelle 4:** Topic Modeling für einschlägige Werke (Tags)  

#### Vorverarbeitung
**Texte:**
- Normalisierung und linguistische Vorverarbeitung (Tokenisierung, Lemmatisierung)  
- Entfernung englischer Stoppwörter und korpusspezifischer Stoppwörter (z. B. Figuren, Sprechverben)  
- Bag-of-Words-Vektorisierung (`CountVectorizer`)  
- Ausschluss sehr seltener (`min_df=2`) und extrem häufiger Wörter (`max_df=0.5`)  

**Tags:**
- Normalisierung (Kleinschreibung, Leerzeichen und Bindestriche → Unterstriche)  
- Keine Lemmatisierung (Tags sind semantisch stabil)  
- Bag-of-Words-Vektorisierung mit tokenisierten Tags  
- Alle Tags berücksichtigt (`min_df=1`)  

#### Topic Modeling
- Methode: **Latent Dirichlet Allocation (LDA)**  
- Jedes Dokument wird als Wahrscheinlichkeitsverteilung über Topics modelliert, jedes Topic als Verteilung über Wörter/Tags  
- Anzahl Topics: `n=3` für Text und Tags (vergleichbar)  
- Batch-Lernverfahren, `random_state=42` (Reproduzierbarkeit)  
- Zusätzlich: **Topic-Dominanz**  
  - Ein Topic gilt als dominant, wenn sein Anteil > 0.5  
  - Ermöglicht die Unterscheidung zwischen klar fokussierten und heterogenen Werken

In [56]:
# --- Topic Modeling für alle Werke (Text) mit optionalen Charakter-Stopwords ---

#---Einstellung: Charakterliste als Stopwords nutzen ----------------------------------
USE_CHARACTER_STOPWORDS = True  # True = Charakter-Namen werden als Stopwords hinzugefügt, False = nicht
analysis_lines.append(f"Charakter-Stopwords kombiniert: {USE_CHARACTER_STOPWORDS}")
#--------------------------------------------------------------------------------------

analysis_lines = []

# Texte aller Werke
texts_all = df["text_topic"].tolist()

# --- Stopwords vorbereiten ---
all_stopwords = set(ENGLISH_STOP_WORDS)
all_stopwords.update(stopword_list)

if USE_CHARACTER_STOPWORDS:
    all_stopwords.update(character_list)

analysis_lines.append(f"Stopword-Liste für Topic Modeling verwendet: {len(all_stopwords)} Begriffe")
analysis_lines.append(f"(Charkter-Namen berücksichtigt: {USE_CHARACTER_STOPWORDS}")

# --- Bag-of-Words ---
vectorizer_all = CountVectorizer(
    ngram_range=(1,2),
    max_df=0.65,
    min_df=5,
    stop_words=list(all_stopwords)
)
X_all = vectorizer_all.fit_transform(texts_all)
analysis_lines.append(f"Vokabulargröße nach Stopword-Filterung: {len(vectorizer_all.get_feature_names_out())}")

# --- LDA-Modell ---
n_topics_all = 10
top_n = 15

lda_all = LatentDirichletAllocation(
    n_components=n_topics_all,
    max_iter=10,
    learning_method='online',
    random_state=42
)
lda_all.fit(X_all)

# --- Topics extrahieren ---
def get_topics(model, vectorizer, top_n=15):
    words = vectorizer.get_feature_names_out()
    topic_words = {}
    for idx, topic in enumerate(model.components_):
        top_words = [words[i] for i in topic.argsort()[-top_n:][::-1]]
        topic_words[f"Topic {idx+1}"] = top_words
    return topic_words

topic_words = get_topics(lda_all, vectorizer_all, top_n)
analysis_lines.append("Topic Modeling – Top Wörter pro Topic:")
analysis_lines.append("")
for t, words in topic_words.items():
    analysis_lines.append(f"{t}: {', '.join(words)}")
    analysis_lines.append("")

# --- Topic-Verteilung pro Dokument ---
doc_topic_all = lda_all.transform(X_all)

# --- Dominanzanalyse ---
dominant_threshold_5 = 0.5
topic_counts_5 = (doc_topic_all > dominant_threshold_5).sum(axis=0)
analysis_lines.append("Anzahl der Werke, in denen die Topics stark dominierend sind (>=0.5):")
for i, count in enumerate(topic_counts_5):
    analysis_lines.append(f"Topic {i+1}: {count} Werke ({round(100*count/len(df),1)}%)")
analysis_lines.append("")

dominant_threshold_3 = 0.3
topic_counts_3 = (doc_topic_all > dominant_threshold_3).sum(axis=0)
analysis_lines.append("Anzahl der Werke, in denen die Topics präsent sind (>=0.3):")
for i, count in enumerate(topic_counts_3):
    analysis_lines.append(f"Topic {i+1}: {count} Werke ({round(100*count/len(df),1)}%)")
analysis_lines.append("")

# --- CSV-Ausgabe der vollständigen Topic-Verteilung ---
output_path = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_topicmodeling_text_all.csv"
)
df_doc_topic_all = pd.DataFrame(
    doc_topic_all,
    columns=[f"Topic_{i+1}" for i in range(n_topics_all)]
)
df_doc_topic_all["work_id"] = df["work_id"].values
df_doc_topic_all.to_csv(output_path, index=False, encoding="utf-8")
analysis_lines.append(f"Vollständige Topic-Verteilung für alle Werke gespeichert unter: {output_path}")
analysis_lines.append("")

# --- Visualisierung: Durchschnittlicher Topic-Anteil ---
topic_means = df_doc_topic_all.drop(columns="work_id").mean().sort_values(ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(x=topic_means.values, y=topic_means.index, palette="viridis")
plt.xlabel("Durchschnittlicher Topic-Anteil")
plt.ylabel("Topic")
plt.title(f"Relevanz der Topics im Korpus für {OUTPUT_PREFIX}")
plt.tight_layout()
topic_means_file = os.path.join(OUTPUT_FOLDER, f"{OUTPUT_PREFIX}_graph_topic_text_all_means.png")
plt.savefig(topic_means_file)
plt.close()
analysis_lines.append(f"Grafik zum durchschnittlichen Topic-Anteil gespeichert unter: {topic_means_file}")

# --- Visualisierung: Dominantes Topic pro Werk ---
dominant_topic = doc_topic_all.argmax(axis=1)
dominant_counts = pd.Series(dominant_topic).value_counts().sort_index()
plt.figure(figsize=(8,5))
sns.barplot(
    x=[f"Topic {i+1}" for i in dominant_counts.index],
    y=dominant_counts.values,
    palette="viridis"
)
plt.ylabel("Anzahl Werke")
plt.xlabel("Dominantes Topic")
plt.title(f"Dominantes Topic pro Werk für {OUTPUT_PREFIX}")
plt.tight_layout()
dominant_topic_file = os.path.join(OUTPUT_FOLDER, f"{OUTPUT_PREFIX}_graph_topic_text_all_dominant.png")
plt.savefig(dominant_topic_file)
plt.close()
analysis_lines.append(f"Grafik zu den dominanten Topics pro Werk gespeichert unter: {dominant_topic_file}")

# --- In Analyse-Dokument schreiben ---
write_analysis_block(
    title="Topic Modeling über alle Werke (Texte, inkl. optionaler Charakter-Stopwords)",
    lines=analysis_lines
)


C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\1590962917.py:96: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=topic_means.values, y=topic_means.index, palette="viridis")
C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\1590962917.py:110: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


[2026-01-10 22:42:34] Topic Modeling über alle Werke (Texte, inkl. optionaler Charakter-Stopwords)
------------------------------------------------------------
Stopword-Liste für Topic Modeling verwendet: 685 Begriffe
(Charkter-Namen berücksichtigt: True
Vokabulargröße nach Stopword-Filterung: 50185
Topic Modeling – Top Wörter pro Topic:

Topic 1: love, friend, man, got, life, need, smile, wanted, people, took, turned, wand, mean, bit, help

Topic 2: barty, evan, marlene, pandora, dorcas, reg, barty evan, reggie, cigarette, evan barty, rosier, pete, princess, sybil, party

Topic 3: dan, cormac, torus, duo, men, fan, ran, fang, chang, ban, ding, rang, man, que, hen

Topic 4: oliver, pumpkin, candy, ginevra, cake, penelope, halloween, broom, puddlemere, perce, patch, pumpkin patch, effie, gourd, walburga

Topic 5: lyall, raised single, did live, started running, cock, body, breath, need, wand, hard, hip, sound, fucking, light, fuck

Topic 6: version, unmasked, love, sound, wanted, breath

In [57]:
# --- Topic Modeling für einschlägige Werke (Text) mit optionalen Charakter-Stopwords ---
#---Einstellung: Charakterliste als Stopwords nutzen ----------------------------------
USE_CHARACTER_STOPWORDS = True  # True = Charakter-Namen werden als Stopwords hinzugefügt, False = nicht
analysis_lines.append(f"Charakter-Stopwords kombiniert: {USE_CHARACTER_STOPWORDS}")
#--------------------------------------------------------------------------------------

analysis_lines = []

# --- Einschlägige Werke filtern ---
df_relevant = df[df["is_relevant"]].copy()
texts_relevant = df_relevant["text_topic"].tolist()

# --- Stopwords vorbereiten ---
all_stopwords = set(ENGLISH_STOP_WORDS)
all_stopwords.update(stopword_list)

if USE_CHARACTER_STOPWORDS:
    all_stopwords.update(character_list)

analysis_lines.append(f"Stopword-Liste für Topic Modeling verwendet: {len(all_stopwords)} Begriffe")
analysis_lines.append(f"(Charakter-Namen berücksichtigt: {USE_CHARACTER_STOPWORDS})")

# --- Bag-of-Words ---
vectorizer_rel = CountVectorizer(
    ngram_range=(1,2),
    max_df=0.65,
    min_df=2,
    stop_words=list(all_stopwords)
)
X_rel = vectorizer_rel.fit_transform(texts_relevant)
analysis_lines.append(f"Vokabulargröße nach Stopword-Filterung: {len(vectorizer_rel.get_feature_names_out())}")

# --- LDA-Modell ---
n_topics_rel = 5
top_n_words = 10

lda_rel = LatentDirichletAllocation(
    n_components=n_topics_rel,
    max_iter=10,
    learning_method='online',
    random_state=42
)
lda_rel.fit(X_rel)

# --- Topics extrahieren ---
def get_topics(model, vectorizer, top_n=10):
    words = vectorizer.get_feature_names_out()
    topic_words = {}
    for idx, topic in enumerate(model.components_):
        top_words = [words[i] for i in topic.argsort()[-top_n:][::-1]]
        topic_words[f"Topic {idx+1}"] = top_words
    return topic_words

topic_words_rel = get_topics(lda_rel, vectorizer_rel, top_n_words)
analysis_lines.append("Topic Modeling – Top Wörter pro Topic (einschlägige Werke):")
analysis_lines.append("")

for t, words in topic_words_rel.items():
    analysis_lines.append(f"{t}: {', '.join(words)}")
    analysis_lines.append("")

# --- Topic-Verteilung pro Dokument ---
doc_topic_rel = lda_rel.transform(X_rel)

# --- Dominanzanalyse ---
dominant_threshold_5 = 0.5
topic_counts_5 = (doc_topic_rel > dominant_threshold_5).sum(axis=0)
analysis_lines.append("Anzahl der Werke, in denen die Topics stark dominant sind (>=0.5):")
for i, count in enumerate(topic_counts_5):
    analysis_lines.append(f"Topic {i+1}: {count} Werke ({round(100*count/len(df_relevant),1)}%)")
analysis_lines.append("")

dominant_threshold_3 = 0.3
topic_counts_3 = (doc_topic_rel > dominant_threshold_3).sum(axis=0)
analysis_lines.append("Anzahl der Werke, in denen die Topics präsent sind (>=0.3):")
for i, count in enumerate(topic_counts_3):
    analysis_lines.append(f"Topic {i+1}: {count} Werke ({round(100*count/len(df_relevant),1)}%)")
analysis_lines.append("")

# --- CSV-Ausgabe der vollständigen Topic-Verteilung ---
output_path_rel = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_topicmodeling_text_rel.csv"
)
df_doc_topic_rel = pd.DataFrame(
    doc_topic_rel,
    columns=[f"Topic_{i+1}" for i in range(n_topics_rel)]
)
df_doc_topic_rel["work_id"] = df_relevant["work_id"].values
df_doc_topic_rel.to_csv(output_path_rel, index=False, encoding="utf-8")
analysis_lines.append(f"Vollständige Topic-Verteilung für einschlägige Werke gespeichert unter: {output_path_rel}")
analysis_lines.append("")

# --- Visualisierung: Durchschnittlicher Topic-Anteil ---
topic_means = df_doc_topic_rel.drop(columns="work_id").mean().sort_values(ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(x=topic_means.values, y=topic_means.index, palette="viridis")
plt.xlabel("Durchschnittlicher Topic-Anteil")
plt.ylabel("Topic")
plt.title(f"Relevanz der Topics (einschlägige Werke) für {OUTPUT_PREFIX}")
plt.tight_layout()
topic_means_file = os.path.join(OUTPUT_FOLDER, f"{OUTPUT_PREFIX}_graph_topic_text_rel_means.png")
plt.savefig(topic_means_file)
plt.close()
analysis_lines.append(f"Grafik zum durchschnittlichen Topic-Anteil gespeichert unter: {topic_means_file}")

# --- Visualisierung: Dominantes Topic pro Werk ---
dominant_topic = doc_topic_rel.argmax(axis=1)
dominant_counts = pd.Series(dominant_topic).value_counts().sort_index()
plt.figure(figsize=(8,5))
sns.barplot(
    x=[f"Topic {i+1}" for i in dominant_counts.index],
    y=dominant_counts.values,
    palette="viridis"
)
plt.ylabel("Anzahl Werke")
plt.xlabel("Dominantes Topic")
plt.title(f"Dominantes Topic pro Werk (einschlägige Werke) für {OUTPUT_PREFIX}")
plt.tight_layout()
dominant_topic_file = os.path.join(OUTPUT_FOLDER, f"{OUTPUT_PREFIX}_graph_topic_text_rel_dominant.png")
plt.savefig(dominant_topic_file)
plt.close()
analysis_lines.append(f"Grafik zu den dominanten Topics pro Werk gespeichert unter: {dominant_topic_file}")

# --- In Analyse-Dokument schreiben ---
write_analysis_block(
    title="Topic Modeling – Zusammenfassung einschlägige Werke (Text)",
    lines=analysis_lines
)


C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\207008260.py:97: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=topic_means.values, y=topic_means.index, palette="viridis")
C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\207008260.py:111: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


[2026-01-10 22:42:36] Topic Modeling – Zusammenfassung einschlägige Werke (Text)
------------------------------------------------------------
Stopword-Liste für Topic Modeling verwendet: 685 Begriffe
(Charakter-Namen berücksichtigt: True)
Vokabulargröße nach Stopword-Filterung: 20214
Topic Modeling – Top Wörter pro Topic (einschlägige Werke):

Topic 1: cock, floor, saw, second, pulled, doe, new, wand, magic, light

Topic 2: oliver, lavender, penelope, friend, past, sat, madam, perce, sorry, broom

Topic 3: cock, dark, light, work, second, magic, new, pulled, world, air

Topic 4: oliver, loved, love oliver, sorry, book, work, kissed, perce, friend, world

Topic 5: van, baby, toy, cock, hip, seat, seatbelt, low, fucking, moan

Anzahl der Werke, in denen die Topics stark dominant sind (>=0.5):
Topic 1: 0 Werke (0.0%)
Topic 2: 3 Werke (3.1%)
Topic 3: 94 Werke (95.9%)
Topic 4: 1 Werke (1.0%)
Topic 5: 0 Werke (0.0%)

Anzahl der Werke, in denen die Topics präsent sind (>=0.3):
Topic 1: 0 Werk

In [58]:
# --- Topic Modeling für alle Werke (Tags) mit optionalen Charakter-Stopwords ---
#---Einstellung: Charakterliste als Stopwords nutzen ----------------------------------
USE_CHARACTER_STOPWORDS = True  # True = Charakter-Namen werden als Stopwords hinzugefügt, False = nicht
analysis_lines.append(f"Charakter-Stopwords kombiniert: {USE_CHARACTER_STOPWORDS}")
#--------------------------------------------------------------------------------------

analysis_lines = []

# --- Alle Werke betrachten ---
df_all = df.copy()

# --- Tags vorbereiten ---
def normalize_tag(tag):
    tag = str(tag).lower().strip()
    tag = tag.replace(" ", "_")
    tag = tag.replace("-", "_")
    return tag

# - Freeform-Tags normalisieren
texts_tags_all = [
    [normalize_tag(t) for t in tags if str(t).lower() != "nan"]
    for tags in df_all["freeform"]
]

# - Nur Werke mit mindestens einem Tag behalten
valid_idx = [i for i, tags in enumerate(texts_tags_all) if len(tags) > 0]
texts_tags_all = [texts_tags_all[i] for i in valid_idx]
work_ids_tags_all = df_all.iloc[valid_idx]["work_id"].values

analysis_lines.append("Topic Modeling für alle Werke auf Basis von Tags.")
analysis_lines.append(f"Anzahl berücksichtigter Werke mit Tags: {len(texts_tags_all)}")
analysis_lines.append("")

# --- Stopwords vorbereiten ---
all_stopwords = set(ENGLISH_STOP_WORDS)
all_stopwords.update(stopword_list)

if USE_CHARACTER_STOPWORDS:
    all_stopwords.update(character_list)

analysis_lines.append(f"Stopword-Liste für Tags verwendet: {len(all_stopwords)} Begriffe")
analysis_lines.append(f"(Charakter-Namen berücksichtigt: {USE_CHARACTER_STOPWORDS})")

# --- Bag-of-Words ---
vectorizer_tags_all = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    min_df=1
)

# - Stopwords filtern
texts_tags_all_filtered = [
    [t for t in tags if t not in all_stopwords] for tags in texts_tags_all
]

X_tags_all = vectorizer_tags_all.fit_transform(texts_tags_all_filtered)

analysis_lines.append(
    f"Anzahl verschiedener Tags im Vokabular nach Stopword-Filterung: {len(vectorizer_tags_all.get_feature_names_out())}"
)
analysis_lines.append("")

# --- LDA-Modell ---
n_topics_tags_all = 5
top_n_words = 5

lda_tags_all = LatentDirichletAllocation(
    n_components=n_topics_tags_all,
    max_iter=30,
    learning_method="batch",
    random_state=42
)
lda_tags_all.fit(X_tags_all)

# --- Topics extrahieren ---
def get_topics(model, vectorizer, top_n):
    words = vectorizer.get_feature_names_out()
    topic_words = {}
    for idx, topic in enumerate(model.components_):
        top_words = [words[i] for i in topic.argsort()[-top_n:][::-1]]
        topic_words[f"Topic {idx+1}"] = top_words
    return topic_words

topic_words_tags_all = get_topics(
    lda_tags_all,
    vectorizer_tags_all,
    top_n_words
)

analysis_lines.append("Top Tags pro Topic (alle Werke):")
analysis_lines.append("")
for topic, words in topic_words_tags_all.items():
    analysis_lines.append(f"{topic}: {', '.join(words)}")
analysis_lines.append("")
analysis_lines.append("")

# --- Topic-Verteilung pro Werk ---
doc_topics_tags_all = lda_tags_all.transform(X_tags_all)

# --- Dominanzanalyse ---
dominant_threshold_5 = 0.5
topic_counts_5 = (doc_topics_tags_all >= dominant_threshold_5).sum(axis=0)

analysis_lines.append("Anzahl der Werke mit dominantem Topic (>= 0.5):")
for i, count in enumerate(topic_counts_5):
    analysis_lines.append(
        f"Topic {i+1}: {count} Werke "
        f"({round(100 * count / len(work_ids_tags_all), 1)}%)"
    )
analysis_lines.append("")

dominant_threshold_3 = 0.3
topic_counts_3 = (doc_topics_tags_all >= dominant_threshold_3).sum(axis=0)

analysis_lines.append("Anzahl der Werke mit präsentem Topic (>= 0.3):")
for i, count in enumerate(topic_counts_3):
    analysis_lines.append(
        f"Topic {i+1}: {count} Werke "
        f"({round(100 * count / len(work_ids_tags_all), 1)}%)"
    )
analysis_lines.append("")

# --- Vollständige Topic-Verteilung speichern ---
df_doc_topics_tags_all = pd.DataFrame(
    doc_topics_tags_all,
    columns=[f"Topic_{i+1}" for i in range(n_topics_tags_all)]
)
df_doc_topics_tags_all["work_id"] = work_ids_tags_all

output_path_tags_all = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_topicmodeling_tags_all.csv"
)
df_doc_topics_tags_all.to_csv(
    output_path_tags_all,
    index=False,
    encoding="utf-8"
)

analysis_lines.append(f"Vollständige Topic-Verteilung (Tags, alle Werke) gespeichert unter:")
analysis_lines.append(output_path_tags_all)
analysis_lines.append("")

# --- Visualisierung: Durchschnittlicher Topic-Anteil ---
topic_means = (
    df_doc_topics_tags_all
    .drop(columns="work_id")
    .mean()
    .sort_values(ascending=False)
)

plt.figure(figsize=(8, 5))
sns.barplot(
    x=topic_means.values,
    y=topic_means.index,
    palette="viridis"
)
plt.xlabel("Durchschnittlicher Topic-Anteil")
plt.ylabel("Topic")
plt.title(f"Tag-Topics – durchschnittliche Relevanz für {OUTPUT_PREFIX}")
plt.tight_layout()

topic_means_file = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_graph_topic_tags_all_means.png"
)
plt.savefig(topic_means_file)
plt.close()

analysis_lines.append(f"Grafik: Durchschnittlicher Topic-Anteil gespeichert unter:")
analysis_lines.append(topic_means_file)
analysis_lines.append("")

# --- Visualisierung: Dominantes Topic pro Werk ---
dominant_topic = doc_topics_tags_all.argmax(axis=1)
dominant_counts = pd.Series(dominant_topic).value_counts().sort_index()

plt.figure(figsize=(8, 5))
sns.barplot(
    x=[f"Topic {i+1}" for i in dominant_counts.index],
    y=dominant_counts.values,
    palette="viridis"
)
plt.ylabel("Anzahl Werke")
plt.xlabel("Dominantes Topic")
plt.title(f"Dominantes Tag-Topic pro Werk für {OUTPUT_PREFIX}")
plt.tight_layout()

dominant_topic_file = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_graph_topic_tags_all_dominant.png"
)
plt.savefig(dominant_topic_file)
plt.close()

analysis_lines.append(f"Grafik: Dominantes Topic pro Werk gespeichert unter:")
analysis_lines.append(dominant_topic_file)
analysis_lines.append("")

# --- In Analyse-Dokument schreiben ---
write_analysis_block(
    title="Topic Modeling – Tags aller Werke (Tags)",
    lines=analysis_lines
)


C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\3850752505.py:154: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\3850752505.py:180: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


[2026-01-10 22:42:46] Topic Modeling – Tags aller Werke (Tags)
------------------------------------------------------------
Topic Modeling für alle Werke auf Basis von Tags.
Anzahl berücksichtigter Werke mit Tags: 2073

Stopword-Liste für Tags verwendet: 685 Begriffe
(Charakter-Namen berücksichtigt: True)
Anzahl verschiedener Tags im Vokabular nach Stopword-Filterung: 8887

Top Tags pro Topic (alle Werke):

Topic 1: kinktober_2025, smut, anal_sex, plot_what_plot/porn_without_plot, oral_sex
Topic 2: fluff, halloween, harry_potter_epilogue_what_epilogue_|_ewe, kinktober_2025, alternate_universe___muggle
Topic 3: kinktober_2025, hp_wlw_reverse_small_bang_2025, digital_art, art, fluff
Topic 4: fluff, angst, hurt/comfort, one_shot, flufftober_2025
Topic 5: explicit_sexual_content, alternate_universe___canon_divergence, established_relationship, harry_potter_epilogue_what_epilogue_|_ewe, smut


Anzahl der Werke mit dominantem Topic (>= 0.5):
Topic 1: 631 Werke (30.4%)
Topic 2: 308 Werke (14.

In [59]:
# --- Topic Modeling für einschlägige Werke (Tags) mit optionalen Charakter-Stopwords ---
#---Einstellung: Charakterliste als Stopwords nutzen ----------------------------------
USE_CHARACTER_STOPWORDS = True  # True = Charakter-Namen werden als Stopwords hinzugefügt, False = nicht
analysis_lines.append(f"Charakter-Stopwords kombiniert: {USE_CHARACTER_STOPWORDS}")
#--------------------------------------------------------------------------------------

analysis_lines = []

# --- Einschlägige Werke filtern ---
df_relevant = df[df["is_relevant"]].copy()

# --- Tags vorbereiten ---
def normalize_tag(tag):
    tag = str(tag).lower().strip()
    tag = tag.replace(" ", "_")
    tag = tag.replace("-", "_")
    return tag

# - Freeform-Tags normalisieren
texts_tags_rel = [
    [normalize_tag(t) for t in tags if str(t).lower() != "nan"]
    for tags in df_relevant["freeform"]
]

# - Nur Werke mit mindestens einem Tag behalten
valid_idx = [i for i, tags in enumerate(texts_tags_rel) if len(tags) > 0]
texts_tags_rel = [texts_tags_rel[i] for i in valid_idx]
work_ids_tags_rel = df_relevant.iloc[valid_idx]["work_id"].values

analysis_lines.append("Topic Modeling für einschlägige Werke auf Basis von Tags.")
analysis_lines.append(f"Anzahl berücksichtigter Werke mit Tags: {len(texts_tags_rel)}")
analysis_lines.append("")

# --- Stopwords vorbereiten ---
all_stopwords = set(ENGLISH_STOP_WORDS)
all_stopwords.update(stopword_list)

if USE_CHARACTER_STOPWORDS:
    all_stopwords.update(character_list)

analysis_lines.append(f"Stopword-Liste für Tags verwendet: {len(all_stopwords)} Begriffe")
analysis_lines.append(f"(Charakter-Namen berücksichtigt: {USE_CHARACTER_STOPWORDS})")

# - Stopwords auf Tags anwenden
texts_tags_rel_filtered = [
    [t for t in tags if t not in all_stopwords] for tags in texts_tags_rel
]

# --- Bag-of-Words ---
vectorizer_tags_rel = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    min_df=1
)

X_tags_rel = vectorizer_tags_rel.fit_transform(texts_tags_rel_filtered)

analysis_lines.append(
    f"Anzahl verschiedener Tags im Vokabular nach Stopword-Filterung: {len(vectorizer_tags_rel.get_feature_names_out())}"
)
analysis_lines.append("")

# --- LDA-Modell ---
n_topics_tags_rel = 5
top_n_words = 5

lda_tags_rel = LatentDirichletAllocation(
    n_components=n_topics_tags_rel,
    max_iter=30,
    learning_method="batch",
    random_state=42
)
lda_tags_rel.fit(X_tags_rel)

# --- Topics extrahieren ---
def get_topics(model, vectorizer, top_n):
    words = vectorizer.get_feature_names_out()
    topic_words = {}
    for idx, topic in enumerate(model.components_):
        top_words = [words[i] for i in topic.argsort()[-top_n:][::-1]]
        topic_words[f"Topic {idx+1}"] = top_words
    return topic_words

topic_words_tags_rel = get_topics(
    lda_tags_rel,
    vectorizer_tags_rel,
    top_n_words
)

analysis_lines.append("Top Tags pro Topic (einschlägige Werke):")
analysis_lines.append("")
for topic, words in topic_words_tags_rel.items():
    analysis_lines.append(f"{topic}: {', '.join(words)}")
analysis_lines.append("")
analysis_lines.append("")

# --- Topic-Verteilung pro Werk ---
doc_topics_tags_rel = lda_tags_rel.transform(X_tags_rel)

# --- Dominanzanalyse ---
dominant_threshold_5 = 0.5
topic_counts_5 = (doc_topics_tags_rel >= dominant_threshold_5).sum(axis=0)

analysis_lines.append("Anzahl der Werke mit dominantem Topic (>= 0.5):")
for i, count in enumerate(topic_counts_5):
    analysis_lines.append(
        f"Topic {i+1}: {count} Werke "
        f"({round(100 * count / len(work_ids_tags_rel), 1)}%)"
    )
analysis_lines.append("")

dominant_threshold_3 = 0.3
topic_counts_3 = (doc_topics_tags_rel >= dominant_threshold_3).sum(axis=0)

analysis_lines.append("Anzahl der Werke mit präsentem Topic (>= 0.3):")
for i, count in enumerate(topic_counts_3):
    analysis_lines.append(
        f"Topic {i+1}: {count} Werke "
        f"({round(100 * count / len(work_ids_tags_rel), 1)}%)"
    )
analysis_lines.append("")

# --- Vollständige Topic-Verteilung speichern ---
df_doc_topics_tags_rel = pd.DataFrame(
    doc_topics_tags_rel,
    columns=[f"Topic_{i+1}" for i in range(n_topics_tags_rel)]
)
df_doc_topics_tags_rel["work_id"] = work_ids_tags_rel

output_path_tags_rel = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_topicmodeling_tags_rel.csv"
)
df_doc_topics_tags_rel.to_csv(
    output_path_tags_rel,
    index=False,
    encoding="utf-8"
)

analysis_lines.append(f"Vollständige Topic-Verteilung (Tags, einschlägige Werke) gespeichert unter:")
analysis_lines.append(output_path_tags_rel)
analysis_lines.append("")

# --- Visualisierung: Durchschnittlicher Topic-Anteil ---
topic_means = (
    df_doc_topics_tags_rel
    .drop(columns="work_id")
    .mean()
    .sort_values(ascending=False)
)

plt.figure(figsize=(8, 5))
sns.barplot(
    x=topic_means.values,
    y=topic_means.index,
    palette="viridis"
)
plt.xlabel("Durchschnittlicher Topic-Anteil")
plt.ylabel("Topic")
plt.title(f"Tag-Topics – durchschnittliche Relevanz für {OUTPUT_PREFIX}")
plt.tight_layout()

topic_means_file = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_graph_topic_tags_rel_means.png"
)
plt.savefig(topic_means_file)
plt.close()

analysis_lines.append(f"Grafik: Durchschnittlicher Topic-Anteil gespeichert unter:")
analysis_lines.append(topic_means_file)
analysis_lines.append("")

# --- Visualisierung: Dominantes Topic pro Werk ---
dominant_topic = doc_topics_tags_rel.argmax(axis=1)
dominant_counts = pd.Series(dominant_topic).value_counts().sort_index()

plt.figure(figsize=(8, 5))
sns.barplot(
    x=[f"Topic {i+1}" for i in dominant_counts.index],
    y=dominant_counts.values,
    palette="viridis"
)
plt.ylabel("Anzahl Werke")
plt.xlabel("Dominantes Topic")
plt.title(f"Dominantes Tag-Topic pro Werk für {OUTPUT_PREFIX}")
plt.tight_layout()

dominant_topic_file = os.path.join(
    OUTPUT_FOLDER,
    f"{OUTPUT_PREFIX}_graph_topic_tags_rel_dominant.png"
)
plt.savefig(dominant_topic_file)
plt.close()

analysis_lines.append(f"Grafik: Dominantes Topic pro Werk gespeichert unter:")
analysis_lines.append(dominant_topic_file)
analysis_lines.append("")

# --- In Analyse-Dokument schreiben ---
write_analysis_block(
    title="Topic Modeling – Tags einschlägiger Werke (inkl. optionaler Charakter-Stopwords)",
    lines=analysis_lines
)


C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\1248677206.py:154: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
C:\Users\Uni\AppData\Local\Temp\ipykernel_5488\1248677206.py:180: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


[2026-01-10 22:42:47] Topic Modeling – Tags einschlägiger Werke (inkl. optionaler Charakter-Stopwords)
------------------------------------------------------------
Topic Modeling für einschlägige Werke auf Basis von Tags.
Anzahl berücksichtigter Werke mit Tags: 98

Stopword-Liste für Tags verwendet: 685 Begriffe
(Charakter-Namen berücksichtigt: True)
Anzahl verschiedener Tags im Vokabular nach Stopword-Filterung: 1027

Top Tags pro Topic (einschlägige Werke):

Topic 1: trans_regulus_black, oral_sex, kinktober_2025, vaginal_sex, anal_sex
Topic 2: kinktober_2025, anal_sex, praise_kink, smut, shameless_smut
Topic 3: kinktober_2025, trans_regulus_black, kinktober, alternate_universe___muggle, one_shot
Topic 4: trans_regulus_black, alternate_universe___modern_setting, plot_what_plot/porn_without_plot, fluff, smut
Topic 5: trans_regulus_black, trans_male_character, vaginal_sex, plot_what_plot/porn_without_plot, oral_sex


Anzahl der Werke mit dominantem Topic (>= 0.5):
Topic 1: 25 Werke (25.